In [1]:
from pyspark.sql import SparkSession

In [2]:
# Build a spark session by analyzing crime report over a period of 8 years
spark = SparkSession.builder\
                    .appName("Analyzing London Crime data")\
                    .getOrCreate()

In [3]:
data = spark.read\
            .format("csv")\
            .option("header","true")\
            .load("../datasets/london_crime_by_lsoa.csv")

In [5]:
data.printSchema()

root
 |-- lsoa_code: string (nullable = true)
 |-- borough: string (nullable = true)
 |-- major_category: string (nullable = true)
 |-- minor_category: string (nullable = true)
 |-- value: string (nullable = true)
 |-- year: string (nullable = true)
 |-- month: string (nullable = true)



In [6]:
data.count()

13490604

In [8]:
# Access a subset of a dataset through limit function
# Value field indicates the number of convictions from that crime report
data.limit(5).show()

+---------+----------+--------------------+--------------------+-----+----+-----+
|lsoa_code|   borough|      major_category|      minor_category|value|year|month|
+---------+----------+--------------------+--------------------+-----+----+-----+
|E01001116|   Croydon|            Burglary|Burglary in Other...|    0|2016|   11|
|E01001646| Greenwich|Violence Against ...|      Other violence|    0|2016|   11|
|E01000677|   Bromley|Violence Against ...|      Other violence|    0|2015|    5|
|E01003774| Redbridge|            Burglary|Burglary in Other...|    0|2016|    3|
|E01004563|Wandsworth|             Robbery|   Personal Property|    0|2008|    6|
+---------+----------+--------------------+--------------------+-----+----+-----+



In [9]:
data.dropna()

DataFrame[lsoa_code: string, borough: string, major_category: string, minor_category: string, value: string, year: string, month: string]

In [10]:
# Remove unnecessary columns
# Unique code for Crime Report
data = data.drop("lsoa_code")
data.limit(5).show()

+----------+--------------------+--------------------+-----+----+-----+
|   borough|      major_category|      minor_category|value|year|month|
+----------+--------------------+--------------------+-----+----+-----+
|   Croydon|            Burglary|Burglary in Other...|    0|2016|   11|
| Greenwich|Violence Against ...|      Other violence|    0|2016|   11|
|   Bromley|Violence Against ...|      Other violence|    0|2015|    5|
| Redbridge|            Burglary|Burglary in Other...|    0|2016|    3|
|Wandsworth|             Robbery|   Personal Property|    0|2008|    6|
+----------+--------------------+--------------------+-----+----+-----+



In [12]:
# Show distinct records
total_boroughs = data.select('borough').distinct()

In [13]:
total_boroughs.show()

+--------------------+
|             borough|
+--------------------+
|             Croydon|
|          Wandsworth|
|              Bexley|
|             Lambeth|
|Barking and Dagenham|
|              Camden|
|           Greenwich|
|              Newham|
|       Tower Hamlets|
|            Hounslow|
|              Barnet|
|              Harrow|
|Kensington and Ch...|
|           Islington|
|               Brent|
|            Haringey|
|             Bromley|
|              Merton|
|         Westminster|
|             Hackney|
+--------------------+
only showing top 20 rows



In [15]:
total_boroughs.count()

33

In [23]:
# Filter records from Hackney Borrow - Like a sql select with where clause
hackney_data = data.filter(data['borough'] == 'Hackney')
hackney_data.show(5)

+-------+--------------------+--------------------+-----+----+-----+
|borough|      major_category|      minor_category|value|year|month|
+-------+--------------------+--------------------+-----+----+-----+
|Hackney|     Criminal Damage|Criminal Damage T...|    0|2011|    6|
|Hackney|Violence Against ...|          Harassment|    1|2013|    2|
|Hackney|     Criminal Damage|Other Criminal Da...|    0|2011|    7|
|Hackney|Violence Against ...|        Wounding/GBH|    0|2013|   12|
|Hackney|  Theft and Handling|  Other Theft Person|    0|2016|    8|
+-------+--------------------+--------------------+-----+----+-----+
only showing top 5 rows



In [24]:
hackney_data.count()

417744

In [27]:
# Filter records of 2015, 2016 - Like a sql in clause
data_2015_2016 = data.filter(data['year'].isin(["2015", "2016"]))
data_2015_2016.show(5)

+---------+--------------------+--------------------+-----+----+-----+
|  borough|      major_category|      minor_category|value|year|month|
+---------+--------------------+--------------------+-----+----+-----+
|  Croydon|            Burglary|Burglary in Other...|    0|2016|   11|
|Greenwich|Violence Against ...|      Other violence|    0|2016|   11|
|  Bromley|Violence Against ...|      Other violence|    0|2015|    5|
|Redbridge|            Burglary|Burglary in Other...|    0|2016|    3|
|   Sutton|  Theft and Handling|Theft/Taking of P...|    1|2016|    8|
+---------+--------------------+--------------------+-----+----+-----+
only showing top 5 rows



In [28]:
# Show a sample by specifying the fraction
data_2015_2016.sample(fraction=0.5).show()

+--------------+--------------------+--------------------+-----+----+-----+
|       borough|      major_category|      minor_category|value|year|month|
+--------------+--------------------+--------------------+-----+----+-----+
|       Croydon|            Burglary|Burglary in Other...|    0|2016|   11|
|     Greenwich|Violence Against ...|      Other violence|    0|2016|   11|
|     Redbridge|            Burglary|Burglary in Other...|    0|2016|    3|
|       Lambeth|Violence Against ...|      Other violence|    0|2015|    4|
|    Hillingdon|  Theft and Handling|Theft/Taking Of M...|    0|2016|    2|
|      Haringey|Violence Against ...|        Wounding/GBH|    0|2015|   12|
|      Lewisham|Violence Against ...|      Common Assault|    0|2016|    2|
|       Bromley|     Criminal Damage|Criminal Damage T...|    1|2016|    4|
|      Haringey|     Criminal Damage|Criminal Damage T...|    0|2016|   12|
|      Havering|    Fraud or Forgery|  Counted per Victim|    0|2015|   11|
|Waltham For

In [30]:
# Filter records from 2014 onwards using >= clause
data_2014_onwards = data.filter(data['year'] >= 2014)
data_2014_onwards.sample(fraction=0.1).show()

+--------------------+--------------------+--------------------+-----+----+-----+
|             borough|      major_category|      minor_category|value|year|month|
+--------------------+--------------------+--------------------+-----+----+-----+
|               Brent|  Theft and Handling|Theft/Taking of P...|    0|2014|    9|
|             Croydon|Violence Against ...|      Common Assault|    1|2014|   11|
|          Hillingdon|Violence Against ...|    Offensive Weapon|    0|2014|    5|
|            Havering|Violence Against ...|          Harassment|    1|2016|    8|
|              Ealing|Other Notifiable ...|    Other Notifiable|    0|2016|   12|
|              Camden|  Theft and Handling|         Other Theft|    1|2014|    1|
|Kingston upon Thames|Other Notifiable ...|      Going Equipped|    0|2015|   12|
|               Brent|Other Notifiable ...|    Other Notifiable|    0|2016|    4|
|          Wandsworth|Violence Against ...|          Harassment|    0|2016|    4|
|          Wands